### Домашняя работа к уроку 1

In [11]:
import numpy as np

**1) Приведите еще примеры метрик для оценки рекомендаций/ранжирования (можно взять из интернета, или ваши знания)**

In [3]:
recommended_list = [143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43] #id товаров
bought_list = [521, 32, 143, 991]

1. Expected reciprocal rank (ERR)  
$$ERR@K = \sum_{k=1}^{K} \frac{1}{k} \cdot p_k \prod_{i=1}^{k-1} (1-p_i)$$
Т.к. $r^{true} \in \{0; 1\}$, то $p = [bought fact]$

In [12]:
def err_at_k(recommended_list, bought_list, k=5):
    recommended_list = np.flip(np.array(recommended_list)[:k])
    bought_list = np.array(bought_list)

    p_stop = []
    flags = np.isin(recommended_list, bought_list)
    p_stop.append(1 * flags[0])  # добавляю 1-й элемент
    opposite_p = [1 - p*1 for p in flags]
    for i in range(1, k):
        p_stop.append(flags[i] * np.prod(np.array(opposite_p[:i])))
    err_list = [(1 / (j + 1)) * p_stop[j] for j in range(k)]
    return sum(err_list)

In [13]:
err_at_k(recommended_list, bought_list)

0.5

2. PFound
$$PFound@K=\sum_{k=1}^{K} pLook(k) \cdot pRel(K),$$
где:  
$pLook(k)=pLook(k-1)(1-pRel(k-1))(1-pBreake)$  
$pRel(k)$ - вероятность того, что k-й документ окажется релевантным  
$pBreake$ - вероятность того, что пользователь прекратит просмотр по внешним причинам  

В статье Яндекс, описывающей данную метрику указано, что они брали значение $pBreake=0.15$. Для своего расчет я также возьму данную величину. Т.к. в данных $r^{true} \in \{0; 1\}$, то для расчета приму $pRel(k) = [bought fact]$

In [14]:
def plook_at_k(recommended_list, bought_list, k=5):
    recommended_list = np.array(recommended_list)[:k]
    bought_list = np.array(bought_list)
    p_break = 0.15
    p_look_i = []
    flags = np.isin(recommended_list, bought_list)
    p_look_i.append(1)
    for i in range(1, k):
        p_look_i.append(p_look_i[i-1] * (1-flags[i-1])*(1-p_break)*flags[i])
    return sum(p_look_i)

In [15]:
plook_at_k(recommended_list, bought_list)

1.0

**2) Доделать все функции, где стоит комментарий "сделать дома"**

#### HitRate@K

In [16]:
def hit_rate_at_k(recommended_list, bought_list, k=5):
    recommended_list = np.array(recommended_list)[:k]
    bought_list = np.array(bought_list)
    
    flags = np.isin(bought_list, recommended_list)
    return(flags.sum() > 0) * 1

$hit\,rate@3=[1,0,0]=1$

In [17]:
hit_rate_at_k(recommended_list, bought_list, k=3)

1

$hit\,rate@6=[1,0,0,1,0,0]=1$

In [18]:
hit_rate_at_k(recommended_list, bought_list, k=6)

1

#### Precision@K

In [19]:
def precision_at_k(recommended_list, bought_list, k=5):
    recommended_list = np.array(recommended_list)[:k]
    bought_list = np.array(bought_list)
    
    flags = np.isin(bought_list, recommended_list)
    return flags.sum() / len(recommended_list)

$presicion@3=\frac{1+0+0}{1+1+1}=\frac{1}{3}\approx 0.333$

In [20]:
precision_at_k(recommended_list, bought_list, k=3)

0.3333333333333333

$presicion@5=\frac{1+0+0+1+0}{1+1+1+1+1}=\frac{5}{5}=0.4$

In [21]:
precision_at_k(recommended_list, bought_list)

0.4

#### Recall@K & Money Recall@K

In [22]:
prices_recommended = [400, 60, 40, 40, 90, 20, 150, 75, 300, 15]
prices_bought = [300, 50, 400, 40]

In [23]:
def recall_at_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    
    flags = np.isin(bought_list, recommended_list)
    return flags.sum() / len(bought_list)

def money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    prices_recommended = np.array(prices_recommended)[:k]
    prices_bought = np.array(prices_bought)
    
    flags = np.isin(recommended_list, bought_list)
    recall = np.dot(flags, prices_recommended) / prices_bought.sum()
    return recall

$recall@k=\frac{1+0+0+1+0+0+0+0+0+0}{1+1+1+1}=\frac{2}{4}=0.5$

In [24]:
recall_at_k(recommended_list,bought_list,k=10)

0.5

$recall@k=\frac{1\cdot400+0\cdot60+0\cdot40+1\cdot40+0\cdot90+0\cdot20+0\cdot150+0\cdot75+0\cdot300+0\cdot15}{300+50+400+40}=\frac{440}{790}\approx 0.557$

In [25]:
money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought, k=10)

0.5569620253164557

#### MAP@k

In [26]:
def ap_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    
    relevant_indexes = np.nonzero(np.isin(recommended_list, bought_list))[0]
    if len(relevant_indexes) == 0:
        return 0
    
    amount_relevant = len(relevant_indexes)
            
    sum_ = sum([precision_at_k(recommended_list, bought_list, k=index_relevant+1) for index_relevant in relevant_indexes])
    return sum_/amount_relevant

In [27]:
recommended_list_3_users = [[143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43], 
                    [1134, 533, 14, 4, 15, 1543, 1, 99, 27, 3345],
                    [991, 3345, 27, 533, 43, 143, 1543, 156, 1134, 11]
                    ]

bought_list_3_users = [[521, 32, 143],  # юзер 1
                       [143, 156, 991, 43, 11], # юзер 2
                       [1,2]] # юзер 3

In [28]:
def map_k(recommended_list, bought_list, k=5):
    ap_k_list = list(map(lambda x, y: ap_k(x, y, k), recommended_list, bought_list))
    result = sum(ap_k_list) / len(bought_list)   
    return result

$MAP@k=\frac{1+0+0}{3}\approx0.333$

In [29]:
map_k(recommended_list_3_users, bought_list_3_users)

0.3333333333333333

#### MRR@k

In [30]:
def find_rank(recommended_list, bought_list):
    recommended_list = np.array(recommended_list)
    bought_list = np.array(bought_list)

    rank = np.nonzero(np.isin(recommended_list, bought_list))[0]
    if len(rank) > 0:
        return rank[0] + 1

def reciprocal_rank(recommended_list, bought_list, k=1):
    rank_list = list(map(lambda x, y: find_rank(x[:k], y), recommended_list, bought_list))
    mrr = [1 / rank if rank else 0 for rank in rank_list]
    return np.array(mrr).mean()

In [31]:
reciprocal_rank(recommended_list_3_users, bought_list_3_users, k=5)

0.3333333333333333

#### NDCG@k

In [38]:
def ndcg_at_k(recommended_list, bought_list, k=5):
    recommended_list = np.array(recommended_list)[:k]
    bought_list = np.array(bought_list)

    flags = np.isin(recommended_list, bought_list)
    dcg_list = [(flag * 1) / np.log2(i + 2) for i, flag in enumerate(flags)]
    dcg = np.array(dcg_list).sum() / k
    ideal_dcg_list = [1 / j if j <= 2 else 1 / np.log2(j + 1) for j in range(1, len(flags) + 1)]
    ideal_dcg = np.array(ideal_dcg_list).sum() / k
    return dcg / ideal_dcg

In [39]:
ndcg_at_k(recommended_list, bought_list)

0.5077769820926182